In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
numpairs = 10000
shape = (numpairs, 2)
hyp = lambda x: math.sin(math.pi * x)
hyp_vec = np.vectorize(hyp)
xpairs = np.random.uniform(-1,1,(numpairs,1, 2))
labels = hyp_vec(xpairs)

xaxis = np.arange(-1,1,0.001) 
xn = xaxis.shape[0]
yaxis = hyp_vec(xaxis)

In [4]:
h1 = {"w": np.array([]), "var_vals": np.array([]), "w_avg": 0}

h1["w"] = np.average(labels, axis=2)

h1["w_avg"] = np.average(h1["w"])
h1["bias_avg"] = np.average(np.square(np.subtract(h1["w_avg"], yaxis)))                   
h1["var_avg"] = np.average(np.square(np.subtract(h1["w"], h1["w_avg"])))

In [5]:
h1

{'w': array([[-0.57381725],
        [ 0.27809656],
        [-0.07528017],
        ...,
        [-0.20853289],
        [ 0.19344369],
        [ 0.07956812]]),
 'var_vals': array([], dtype=float64),
 'w_avg': -0.0001402504010962854,
 'bias_avg': 0.5000000196701746,
 'var_avg': 0.2527198153241266}

In [6]:
h2 = {"w": np.array([]), "var_vals": np.array([]), "w_avg": 0}

for i in range(numpairs):
    x = xpairs[i].T
    x_pinv = np.linalg.pinv(x)
    y = labels[i].T
    w = np.dot(x_pinv, y).flatten()
    h2["w"] = np.concatenate((h2["w"],w),0)
    
h2["w_avg"] = np.average(h2["w"])
h2["bias_avg"] = np.average(np.square(np.subtract(np.multiply(h2["w_avg"],xaxis),yaxis))) 

h2["gx"] = np.multiply(np.reshape(h2["w"], (numpairs, 1)), np.reshape(xpairs, (numpairs, 2)))
h2["gbx"] = np.multiply(h2["w_avg"], np.reshape(xpairs, (numpairs, 2)))
h2["var_vals"] = np.average(np.square(np.subtract(h2["gx"], h2["gbx"])),axis = 1)

h2["var_avg"] = np.average(h2["var_vals"])

In [7]:
h2

{'w': array([0.60736971, 0.8721874 , 2.58770919, ..., 0.22325259, 2.2956398 ,
        2.83956258]),
 'var_vals': array([0.40966847, 0.14203721, 0.1437421 , ..., 1.25042418, 0.09616079,
        0.11209928]),
 'w_avg': 1.4215933656288613,
 'bias_avg': 0.26862920207337404,
 'gx': array([[-0.3623259 , -0.56976513],
        [-0.74774179,  0.39598367],
        [-0.95036911,  0.71586795],
        ...,
        [-0.20658378, -0.21005508],
        [-0.47399561,  1.04976624],
        [-0.56637526,  0.76046491]]),
 'gbx': array([[-0.84805035, -1.33357708],
        [-1.2187573 ,  0.64542064],
        [-0.52209824,  0.39327183],
        ...,
        [-1.3154523 , -1.33755628],
        [-0.29352558,  0.65007617],
        [-0.28354906,  0.38071775]]),
 'var_avg': 0.19937357799297611}

In [9]:
h3 = {"w": np.array([]), "var_vals": np.array([]), "w_avg": 0}

for i in range(numpairs):
    x = xpairs[i].T
    x_num = x.shape[0] 
    x = np.c_[np.ones(x_num), x]
    x_pinv = np.linalg.pinv(x)
    y = labels[i].T
    w = np.dot(x_pinv, y).flatten()
    h3["w"] = np.concatenate((h3["w"],w))
    
h3["w"] = np.reshape(h3["w"], shape)
h3["w_avg"] = np.reshape(np.average(h3["w"], axis=0), (1,2))
h3["bias_avg"] = np.average(np.square(np.subtract(yaxis, np.dot(h3["w_avg"], np.c_[np.ones(xn), xaxis].T))))                             

for i in range(numpairs):
    x = xpairs[i].T
    x_m = np.c_[np.ones(x.shape[0]), x].T
    w = h3["w"][i]
    w = np.reshape(w, (1, 2))
    gx = np.dot(w, x_m)
    gbar_x = np.multiply(h3["w_avg"], x_m)
    var = np.average(np.square(np.subtract(gx, gbar_x)))
    h3["var_vals"] = np.concatenate((h3["var_vals"],[var]))

h3["var_avg"] = np.average(h3["var_vals"])

In [10]:
h3

{'w': array([[-2.28367592, -2.22835825],
        [ 0.49691712,  1.08513507],
        [ 0.04280171,  2.60605599],
        ...,
        [-3.0752164 , -3.07217871],
        [-0.10792592,  2.40316556],
        [-0.01785067,  2.85050482]]),
 'var_vals': array([0.75138673, 0.20893175, 0.48921888, ..., 0.32492829, 0.29368559,
        0.2410903 ]),
 'w_avg': array([[-0.01714084,  0.76621436]]),
 'bias_avg': 0.20821504516965125,
 'var_avg': 0.5767466551761723}

In [11]:
h4 = {"w": np.array([]), "var_vals": np.array([]), "w_avg": 0}

for i in range(numpairs):
    x = np.square(xpairs[i].T)
    x_pinv = np.linalg.pinv(x)
    y = labels[i].T
    w = np.dot(x_pinv, y).flatten()
    h4["w"] = np.concatenate((h4["w"],w),0)
    
h4["w_avg"] = np.average(h4["w"])
h4["bias_avg"] = np.average(np.square(np.subtract(np.multiply(h4["w_avg"],np.square(xaxis)),yaxis))) 

h4["gx"] = np.multiply(np.reshape(h4["w"], (numpairs, 1)), np.reshape(np.square(xpairs), (numpairs, 2)))
h4["gbar_x"] = np.multiply(h4["w_avg"], np.reshape(np.square(xpairs), (numpairs, 2)))
h4["var_vals"] = np.average(np.square(np.subtract(h4["gx"], h4["gbar_x"])),axis = 1)

h4["var_avg"] = np.average(h4["var_vals"])

In [12]:
h4

{'w': array([-0.56568916, -0.19659777, -2.69743338, ..., -0.23896138,
         3.98453934,  4.48095997]),
 'var_vals': array([0.18241769, 0.02080521, 0.09213767, ..., 0.0726915 , 0.34884614,
        0.06542224]),
 'w_avg': 0.07062713251887386,
 'bias_avg': 0.5009976400322977,
 'gx': array([[-0.20131232, -0.49780967],
        [-0.14449814, -0.04052411],
        [-0.36383495, -0.20643609],
        ...,
        [-0.20461014, -0.21154417],
        [ 0.16987101,  0.8332126 ],
        [ 0.17826937,  0.32138569]]),
 'gbar_x': array([[0.02513414, 0.06215228],
        [0.05191051, 0.01455816],
        [0.00952632, 0.00540513],
        ...,
        [0.06047432, 0.06252374],
        [0.00301101, 0.01476894],
        [0.00280981, 0.00506556]]),
 'var_avg': 0.26225815633824634}

In [18]:
h5 = {"w": np.array([]), "var_vals": np.array([]), "w_avg": 0}

for i in range(numpairs):
    x = xpairs[i].T
    x_num = x.shape[0]
    x = np.c_[np.ones(x_num), np.square(x)]
    x_pinv = np.linalg.pinv(x)
    y = labels[i].T
    w = np.dot(x_pinv, y).flatten()
    h5["w"] = np.concatenate((h5["w"],w))
    
h5["w"] = np.reshape(h5["w"], shape)
h5["w_avg"] = np.average(h5["w"], axis=0)


h5["bias_avg"] = np.average(np.square(np.subtract(yaxis, np.dot(h5["w_avg"], np.c_[np.ones(xn), np.square(xaxis)].T))))                             

for i in range(numpairs):
    x = xpairs[i].T
    x_m = np.c_[np.ones(x.shape[0]), np.square(x)].T
    w = h5["w"][i]
    w = np.reshape(w, (1, 2))
    gx = np.dot(w, x_m)
    gbar_x = np.multiply(h5["w_avg"], x_m)
    var = np.average(np.square(np.subtract(gx, gbar_x)))
    h5["var_vals"] = np.concatenate((h2["var_vals"],[var]))

h5["var_avg"] = np.average(h2["var_vals"])

In [19]:
h5

{'w': array([[ -1.47109092,   1.45204413],
        [  1.5441874 ,  -2.69060211],
        [  2.96458837, -28.75770395],
        ...,
        [ -1.64197414,   1.64620231],
        [ -1.01261085,   9.58159761],
        [ -2.24552353,  41.70334079]]),
 'var_vals': array([ 0.40966847,  0.14203721,  0.1437421 , ...,  0.09616079,
         0.11209928, 19.67394293]),
 'w_avg': array([ 0.47185381, -7.94237109]),
 'bias_avg': 10.84049214423335,
 'var_avg': 0.19937357799297611}